## Part b): Adding Ridge regression for the Franke function

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append("../")
import functions as f 
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
#Using a standardized dataset 
np.random.seed(42)
N = 50
x = np.random.rand(N)
y = np.random.randn(N)
z = f.FrankeFunction(x,y) #Using x,y and z when doing regression
z = z + np.random.normal(0, 0.1, z.shape) #the noise was too high, tried sligtly less

#x = np.arange(0, 1, 0.05)
#y = np.arange(0, 1, 0.05)
#z = f.FrankeFunction(x, y)
#z = z + np.random.randn(z.shape[0])
#z = z + np.random.normal(0, 0.1, z.shape[0])


#Creating lambda values avoiding "b" x.Tin the name as it is a reserved keyword in python
#lamda = np.array([0.001,0.01,0.1, 1])#
lamda = np.logspace(-5, 1, 7)
#lamda = np.logspace(-3, -1, 5)

# Polynomial degrees
#degrees = np.arange(0, 5)
degrees = np.arange(0, 10)

In [ ]:

beta_Ridge_values = []
mse_Ridge_scores = []
r2_Ridge_scores = []

# Looping through each lambda
for i in range(np.size(lamda)):

    # Appending lists to store scores and parameters
    beta_Ridge_values.append([])
    mse_Ridge_scores.append([])
    r2_Ridge_scores.append([])

    # Looping through each degree
    for degree in degrees:
        # Creating design matrix
        X = f.create_design_matrix(x, y, degree)
        
        # Split the data into training and test data
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=42)

        # Scale and center the data
        scaler = StandardScaler(with_std=True, with_mean=False)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        lamda_i = lamda[i]

        betaRidge = f.beta_ridge(X_train, z_train, lamda_i)
        
        ztilde = f.z_predict(X_train, betaRidge)

        mse = f.mse(z_train, ztilde)
        r2 = f.r2(z_train, ztilde)

        beta_Ridge_values[i].append(betaRidge)
        mse_Ridge_scores[i].append(mse)
        r2_Ridge_scores[i].append(r2)



In [ ]:
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree')
#plt.grid()

plt.subplot(1, 2, 2)
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree')
#plt.grid()

for j in range(np.size(lamda)):
    plt.subplot(1, 2, 1)
    plt.plot(degrees, (mse_Ridge_scores[j]), label=f'Lambda = {lamda[j]}')
    plt.subplot(1, 2, 2)
    plt.plot(degrees, r2_Ridge_scores[j], label=f'Lambda = {lamda[j]}')

plt.legend()
plt.tight_layout()
plt.show()

# STUDY THE DEPENDENCE ON LAMBDAS

I think we need a heatmap in plot below

In [ ]:
# Plotting beta values
plt.figure(figsize=(10, 5))
plt.xlabel('Polynomial Degree')
plt.ylabel('Beta Values')
plt.title('Beta Values as a function of Polynomial Degree')

colors = ['red', 'blue', 'green', 'orange', 'purple']  # Define colors for each lambda


for i in range(len(beta_Ridge_values)):
    for j in range(len(beta_Ridge_values[i])):
        for k in range(len(beta_Ridge_values[i][j])):
            beta_i = beta_Ridge_values[i][j][k]
            plt.plot(degrees[j]*np.ones_like(beta_Ridge_values[i][j][k]), beta_Ridge_values[i][j][k], marker='o', label= f'Lambda = {lamda[i]}' if k == 0 else None)  # Use color based on lambda  color=colors[i],

plt.tight_layout()
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label=f'Lambda = {lamda[i]}', markersize=8) for i in range(len(lamda))]) #, markerfacecolor=colors[i]
plt.show()


We can see that the beta values are approximating zero with higher polynomial order. 